BUGS:
If the Course_ID column in the spreadsheet is the same length as the Course_Name Column, the while loops cause problems. Need to add something that prevents the while loop from causing an error if it tries to check for a row that does not exist.

Step 1:
Google Spreadsheet must be shared with the ***REMOVED*** email address.

Step 2:
This page will help with creating the csv files for canvas. 
http://guides.instructure.com/m/4214/l/164118-how-do-i-format-csv-text-files-for-uploading-data-into-canvas
https://canvas.instructure.com/doc/api/file.sis_csv.html


IDEAS:
The listGoogleSheets function has a way to list all the sheets shared to the crazy Gmail address above. This is how the script could then cycle through all the spreadsheets and add stuff to the files for Canvas.

Creating Empty Dataframe that will be used for the CSV exports
http://technicaltidbit.blogspot.com/2013/06/create-empty-dataframe-in-pandas.html

Actually, problem what to just set up columns and then append a row onto each 


File Formats
courses.csv

course_id,	short_name, 	long_name,	status,	account_id

enrollments.csv

course_id,	user_id,	  role,	 status


Over all plan:
For each row in the Google spreadsheet, iterate each cell. Since a user_id will be needed and not their login_id we hopefully can use the Canvas api (https://canvas.instructure.com/doc/api/users.html#User) to look up that information Each cell will then append a row to either the courses or enrollments data frame. At the very end the dataframes converted to csv files.


In [5]:
#Code to import all of the functions that we will need
from __future__ import print_function
import gspread
from oauth2client.client import SignedJwtAssertionCredentials
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import csv

#code to connect to Google Spreadsheets
SECRETS_FILE = "C:\Temp\DataAnalytics-2be8872b079f.json"
SCOPE = ["https://spreadsheets.google.com/feeds"]
json_key = json.load(open(SECRETS_FILE))
# Authenticate using the signed key
#credentials = SignedJwtAssertionCredentials(json_key['client_email'],
#                                           json_key['private_key'], SCOPE)
credentials = SignedJwtAssertionCredentials(json_key['client_email'],
bytes(json_key['private_key'], 'UTF-8'),
SCOPE)

gc = gspread.authorize(credentials)

#CSV Formats
columnsCoursesCSV = ["course_id", "long_name", "short_name", "account_id", "status"]
columnsEnrollmentCSV = ["course_id","user_id","role","status"]

#Define location of file exports. Must have a trailing "/" for it to work.
csvExportLocation = "G:/Inbox/"


#The first group of funtions are Skyward File injestions. Long term we want the script here to access the files from
#a server that has fresh files each night.

#csvfilelocation =  "N:\NHSIT\SkywardtoCanvasExports\SkywardtoCanvasExports"
#csvfilelocation =  "//nhs00/nhsstaffresources/NHSIT/SkywardtoCanvasExports"


def listGoogleSheets():
    """
    This function will just print out the list of Google Spreadsheets in the folder that the json file is pointing to.
    """
    #Code to print out the Spreadsheets that are Available in Google
    print("The following sheets are available")
    for sheet in gc.openall():
        print("{} - {}".format(sheet.title, sheet.id))
    return("That's the List!")


def importSheet(workbookName):
    """
    Given a string that is the workbookName, the first sheet in the workbook will be imported into Python as a dataframe
    """
    workbook = gc.open(workbookName)
    # Get the first sheet
    firstSheet = workbook.sheet1
    # Extract all data into a dataframe
    firstSheetDataFrame = pd.DataFrame(firstSheet.get_all_records())
    lowercaseColumnNames = [x.lower() for x in firstSheetDataFrame.columns.values]
    firstSheetDataFrame.columns = lowercaseColumnNames
    return firstSheetDataFrame

def coursesDF(workbookName,accountID):
    """
    Inputs: Name of a Google Spreadsheet that has been shared with the account whose Oauth Credentials are used in this script
    and an accountID for canvas.
    
    Variables needed for this function:
    Global Variables:
    columnsCoursesCSV: list of the column names needed for the Canvas import
    
    Helper Functions:
    importSheet()
    
    Returns: dataframe called enrollments
    """
    importedGoogleSheet = importSheet(workbookName)
    index = 0
    courses = pd.DataFrame(data=np.zeros((0,len(columnsCoursesCSV))), columns=columnsCoursesCSV)
    while importedGoogleSheet.course_name[index] != '':
        courses = courses.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'long_name':importedGoogleSheet.loc[index,"course_name"],'short_name':importedGoogleSheet.loc[index,"course_name"], 'account_id':accountID, 'status':'active'},ignore_index=True)
        index += 1
    return courses

def enrollmentsDF(workbookName):
    """
    Inputs: Name of a Google Spreadsheet that has been shared with the account whose Oauth Credentials are used in this script.
        
    Variables needed for this function:
    Global Variables:
    lowercaseColumnNames: this contains a list of the column names from the imported Google spreadsheet where all letters are converted to lowercase
    columnsEnrollmentCSV: list of the column names needed for the Canvas import
    
    Helper Functions:
    importSheet()
    
    Returns: dataframe called enrollments
    
    """
    index = 0
    importedGoogleSheet = importSheet(workbookName)
    
    lowercaseColumnNames = importedGoogleSheet.columns
    #create an empty dataframe based on the required columns for a Canvas csv.
    enrollments = pd.DataFrame(data=np.zeros((0,len(columnsEnrollmentCSV))), columns=columnsEnrollmentCSV)
        
    #For enrollments to be created for a course, the course must have a name given in the Google spreadsheet. Next line checks for blank course names.
    while importedGoogleSheet.course_name[index] != '':
        #Three sections of code to add principals (which get added as teachers), teachers, and then students
        #The range for each section is based on the number of columns in the original google sheet
        #Each section has a loop that goes through possible column names that might be in the Google spreadsheet
            #The first if determines if a column name exists in the Google spreadsheet.
                #The second if checks to make sure that that row has a person in that column 
                #A new row is added to the enrollments dataframe if both conditions are met for the possible column names
        
        for x in range(1,len(importedGoogleSheet.columns)):
            possibleColumnName = "principal" + str(x)
            if possibleColumnName in lowercaseColumnNames:
                try:
                    sis_id = canvasStaffandStudentsIndexed.loc[importedGoogleSheet.loc[index,possibleColumnName],'user_id']
                    enrollments = enrollments.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'user_id':sis_id,'role':'Teacher',  'status':'active'},ignore_index=True)
                except: 
                    if importedGoogleSheet.loc[index,possibleColumnName] != '': 
                        enrollments = enrollments.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'user_id':999998,'role':'Student',  'status':'active'},ignore_index=True)
        
        for x in range(1,len(importedGoogleSheet.columns)):
            possibleColumnName = "teacher" + str(x)
            if possibleColumnName in lowercaseColumnNames:
                try:
                    sis_id = canvasStaffandStudentsIndexed.loc[importedGoogleSheet.loc[index,possibleColumnName],'user_id']
                    enrollments = enrollments.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'user_id':sis_id,'role':'Teacher',  'status':'active'},ignore_index=True)
                except:
                    if importedGoogleSheet.loc[index,possibleColumnName] != '':
                        enrollments = enrollments.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'user_id':999998,'role':'Student',  'status':'active'},ignore_index=True)
        
        for x in range(1,len(importedGoogleSheet.columns)):
            possibleColumnName = "student" + str(x)
            if possibleColumnName in lowercaseColumnNames:
                try:
                    sis_id = canvasStaffandStudentsIndexed.loc[importedGoogleSheet.loc[index,possibleColumnName],'user_id']
                    enrollments = enrollments.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'user_id':sis_id,'role':'Student',  'status':'active'},ignore_index=True)
                except:
                    if importedGoogleSheet.loc[index,possibleColumnName] != '':
                        enrollments = enrollments.append({'course_id':importedGoogleSheet.loc[index,"course_id"], 'user_id':999998,'role':'Student',  'status':'active'},ignore_index=True)
        
        #index counter takes us through each row of the dataframe
        index +=1
    
    
    return enrollments


def createCSVsCoursesEnrollmentsTerms(workbookName, AccountID):
    """
    This fuction will take in Google Spreadsheet name.
    Return: A list of lists composed of each row of the spread
    """
    terms = importSheet("terms")
    terms.to_csv(csvExportLocation +"terms.csv", index=False, float_format='%.0f')
    enrollments = enrollmentsDF(workbookName)
    courses = coursesDF(workbookName, AccountID)
    enrollments.to_csv(csvExportLocation +"enrollments.csv", index=False, float_format='%.0f')
    courses.to_csv(csvExportLocation +"courses.csv", index=False, float_format='%.0f')
    return print("Done with CSV Files. Files located in " + csvExportLocation)

     
#These lines import some spreadsheets from Google that are imported from Skyward each day.
#The imports first get a student list and then a staff list and then removed duplicates
#The two dataframes are then combined into one so that any lookups of the student user_id can be made from one dataframe
canvasStudentImport = importSheet("1801Student")
canvasStudentImport.drop_duplicates(inplace = True)
canvasStudentIndexed = canvasStudentImport.set_index(['login_id'])

canvasStaffImport = importSheet("1801Staff")
canvasStaffImport.drop_duplicates(inplace = True)
canvasStaffIndexed = canvasStaffImport.set_index(['login_id'])



canvasStaffandStudentsIndexed = pd.concat([canvasStudentIndexed, canvasStaffIndexed], axis=0)



In [6]:
### Code to import CSV files into Canvas
import requests
import json
csvExportLocation = "G:/Inbox/"
token = "***REMOVED***"
account_id = 1
base_api_url = 'https://ecasd.test.instructure.com/api/'


### Post csv or zip file for Sis Import. Just change the file name.
def sisIMPORT():
    path = 'v1/accounts/{account_id}/sis_imports'
    url = base_api_url + path.format(account_id=account_id)
    files = {'attachment': open('G:/Inbox/courses.csv', 'rb')}
    headers = {'Authorization':'Bearer {token}'.format(token=token)}
    res = requests.post(url, files=files, headers = headers)

In [7]:
createCSVsCoursesEnrollmentsTerms("All Sherman RTI", 180)

KeyError: 6

In [ ]:
sisIMPORT()